# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [75]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import gmaps
import os
import matplotlib.pyplot as plt
import json
import time


# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

# Retrieve Google Places API from api_keys
from api_keys import geoapify_key

In [76]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,40.658,75,75,11.83,AR,1669704740
1,1,ponta do sol,32.6667,-17.1000,66.830,73,23,1.70,PT,1669704947
2,2,lagoa,39.0500,-27.9833,63.968,89,0,9.51,PT,1669704947
3,3,georgetown,5.4112,100.3354,87.746,74,20,4.12,MY,1669704698
4,4,terrace,54.5163,-128.6035,17.510,67,20,0.00,CA,1669704778


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [77]:
# Configure the map plot
gmaps.configure(api_key=geoapify_key)

# Display the map
# Store latitude and longitude in locations
locations = city_data_df[['Lat', 'Lng']]

# Fill NaN values Convert to float
# humid = city_data_df.astype({"Humidity": "float"})
# humid = city_data_df["Humidity"].dropna()
humid = city_data_df.fillna({"Humidity": ''}).astype({"Humidity": "float"})
#lat_lon.head()
# Convert to float



In [78]:
humid

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,40.658,75.0,75,11.83,AR,1669704740
1,1,ponta do sol,32.6667,-17.1000,66.830,73.0,23,1.70,PT,1669704947
2,2,lagoa,39.0500,-27.9833,63.968,89.0,0,9.51,PT,1669704947
3,3,georgetown,5.4112,100.3354,87.746,74.0,20,4.12,MY,1669704698
4,4,terrace,54.5163,-128.6035,17.510,67.0,20,0.00,CA,1669704778
...,...,...,...,...,...,...,...,...,...,...
591,591,pierre,44.3683,-100.3510,27.860,57.0,100,4.92,US,1669705189
592,592,oriximina,-1.7656,-55.8661,73.616,95.0,100,1.79,BR,1669705189
593,593,oyama,36.3000,139.8000,69.584,69.0,84,7.01,JP,1669705189
594,594,buritizeiro,-17.3511,-44.9622,68.252,94.0,45,0.91,BR,1669705190


In [28]:
# Plot & Display the Heatmap
figure_layout = {
    'width': '1200px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2.20 and a center point so the map displays properly at
# a readable size
fig = gmaps.figure(layout=figure_layout, zoom_level=2.20, center=(0, 0))
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid,
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [79]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
# Drop any rows with null values

city_criteria_df = city_data_df.loc[(city_data_df["Wind Speed"] <= 10) & (city_data_df["Cloudiness"] == 0) &
                                    (city_data_df["Max Temp"] >= 70) & (city_data_df["Max Temp"] <= 80)].dropna()

city_criteria_df = city_criteria_df.reset_index()

#  Display sample data
city_criteria_df


,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,19,19,cape town,-33.9258,18.4232,78.890,54,0,6.69,ZA,1669704691
1,52,52,nelson bay,-32.7167,152.1500,70.088,68,0,6.71,AU,1669704832
2,61,61,broken hill,-31.9500,141.4333,74.570,26,0,6.43,AU,1669704971
3,99,99,gaya,24.7833,85.0000,76.910,47,0,0.00,IN,1669704987
4,122,122,carnarvon,-24.8667,113.6333,78.638,66,0,7.62,AU,1669704996
5,153,153,tura,25.5198,90.2201,78.962,44,0,1.33,IN,1669705009
6,208,208,najran,17.4924,44.1277,70.448,26,0,4.83,SA,1669704751
7,240,240,east london,-33.0153,27.9116,74.336,64,0,7.20,ZA,1669705045
8,245,245,forbes,-33.3833,148.0167,77.918,32,0,4.68,AU,1669705047
9,309,309,pampierstad,-27.7832,24.6877,77.918,44,0,6.99,ZA,1669705074


### Step 3: Create a new DataFrame called `hotel_df`.

In [80]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_criteria_df.copy()
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]
hotel_df

,City,Country,Lat,Lng,Humidity
0,cape town,ZA,-33.9258,18.4232,54
1,nelson bay,AU,-32.7167,152.1500,68
2,broken hill,AU,-31.9500,141.4333,26
3,gaya,IN,24.7833,85.0000,47
4,carnarvon,AU,-24.8667,113.6333,66
5,tura,IN,25.5198,90.2201,44
6,najran,SA,17.4924,44.1277,26
7,east london,ZA,-33.0153,27.9116,64
8,forbes,AU,-33.3833,148.0167,32
9,pampierstad,ZA,-27.7832,24.6877,44


In [81]:
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Google Places API
hotel_df = hotel_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
0,cape town,ZA,-33.9258,18.4232,54,
1,nelson bay,AU,-32.7167,152.1500,68,
2,broken hill,AU,-31.9500,141.4333,26,
3,gaya,IN,24.7833,85.0000,47,
4,carnarvon,AU,-24.8667,113.6333,66,
5,tura,IN,25.5198,90.2201,44,
6,najran,SA,17.4924,44.1277,26,
7,east london,ZA,-33.0153,27.9116,64,
8,forbes,AU,-33.3833,148.0167,32,
9,pampierstad,ZA,-27.7832,24.6877,44,


### Step 4: For each city, use the Google Places API to find the first hotel for each city located within 5,000 meters of your coordinates.

In [82]:


# Set parameters to search for a hotel
params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key": geoapify_key}




# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude, and then city name from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Parameters with the current city's latitude and longitude
    params["location"] = f"{lat},{lng}"


    # Set base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

     # Make and API request using the params dictionaty

    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    results = name_address['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cape town - nearest hotel: The Winchester Boutique Hotel Cape Town
nelson bay - nearest hotel: Ramada Resort by Wyndham Shoal Bay
broken hill - nearest hotel: Royal Exchange Hotel
gaya - nearest hotel: Hotel Manisha International
carnarvon - nearest hotel: Carnarvon Motel
tura - nearest hotel: Hotel Polo Orchid
najran - nearest hotel: No hotel found
east london - nearest hotel: Garden Court East London
forbes - nearest hotel: The Plainsman Motel
pampierstad - nearest hotel: No hotel found
sao filipe - nearest hotel: Hotel Santos Pina
orapa - nearest hotel: No hotel found
narsimhapur - nearest hotel: Hotel Kusum Valley and Restaurant
jaciara - nearest hotel: Hotel Toquinho


,City,Country,Lat,Lng,Humidity,Hotel Name
0,cape town,ZA,-33.9258,18.4232,54,The Winchester Boutique Hotel Cape Town
1,nelson bay,AU,-32.7167,152.1500,68,Ramada Resort by Wyndham Shoal Bay
2,broken hill,AU,-31.9500,141.4333,26,Royal Exchange Hotel
3,gaya,IN,24.7833,85.0000,47,Hotel Manisha International
4,carnarvon,AU,-24.8667,113.6333,66,Carnarvon Motel
5,tura,IN,25.5198,90.2201,44,Hotel Polo Orchid
6,najran,SA,17.4924,44.1277,26,No hotel found
7,east london,ZA,-33.0153,27.9116,64,Garden Court East London
8,forbes,AU,-33.3833,148.0167,32,The Plainsman Motel
9,pampierstad,ZA,-27.7832,24.6877,44,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [84]:

# Using the template add the hotel marks to the heatmap
info_box_template = """

Name
{Hotel Name}

City
{City}
Country
{Country}

"""

hotel_data = [info_box_template.format(**row)
              for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [85]:
# Add marker layer ontop of heat map

figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, zoom_level=2, center=(15, 25))

# Create hotel symbol layer
hotel_layer = gmaps.marker_layer(
    locations, info_box_content=[info_box_template.format(
        **row) for index, row in hotel_df.iterrows()]
)


# Add layer
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)
# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…